## Analysing output of PrEWRunRK fits

Using output of test toy fits from PrEW, import it into the PrOut python classes and create some histograms with it.

### Import PrEW output module

In [1]:
import sys
sys.path.append("/afs/desy.de/group/flc/pool/beyerjac/TGCAnalysis/PrEW/source/prout")
import PrOut

### Import other modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt

### Read output file

In [3]:
reader = PrOut.Reader("../output/fit_results.out")
reader.read()
print("Found " + str(len(reader.run_results)) + " run(s).")

Found 1 run(s).


### Collect parameters

In [4]:
# Assuming only one toy run (with many toys) was performed, get number of parameters and performed toy fits
result = reader.run_results[0]
n_pars = len(result.par_names)
n_fits = len(result.fit_results)

# Get the chi-squared values, parameters, uncertainties and correlation matrices from each fit
chi_sqs = np.empty(n_fits)
par_results = np.empty((n_pars,n_fits))
unc_results = np.empty((n_pars,n_fits))
cor_matrix_results = np.empty((n_pars,n_pars,n_fits))
for f in range(n_fits):
    chi_sqs[f] = result.fit_results[f].chisq_fin
    for p in range(n_pars):
        par_results[p][f] = result.fit_results[f].pars_fin[p]
        unc_results[p][f] = result.fit_results[f].uncs_fin[p]
        for p2 in range(n_pars):
            cor_matrix_results[p][p2][f] = result.fit_results[f].cor_matrix[p][p2]

### Calculate result averages

In [5]:
chi_sq_avg = np.average(chi_sqs)
par_avg = [np.average(p_vals) for p_vals in par_results]
unc_avg = [np.average(p_uncs) for p_uncs in unc_results]
cor_matrix_avg = np.array([[np.average(p1p2_element) for p1p2_element in p1_column] for p1_column in cor_matrix_results] )

### Create histograms for each parameter using boost-histogram

In [ ]:
for p in range(n_pars):
    p_avg = par_avg[p] # Average result of current parameter
    p_min = np.amin(par_results[p]) # Smallest result for current parameter
    p_max = np.amax(par_results[p]) # Largest result for current parameter
    p_std = np.std(par_results[p],ddof=1) # Standard deviation of results for current parameter over all fits
    
    # x-axis minimum and maximum
    x_min = p_min - 0.1*(p_avg-p_min)
    x_max = p_max + 0.1*(p_max-p_avg)

    # Create a histogram for this parameter
    fig, ax = plt.subplots(tight_layout=True)
    hist_entries, hist_edges, _ = ax.hist(par_results[p], range=(x_min,x_max), bins=10, histtype='step', color='black', label="Fit results")
    ax.set_xlabel("Fit result")
    ax.set_ylabel("#Toys")
    
    # Calculate bin centers
    bin_centers = 0.5 * (hist_edges[:-1] + hist_edges[1:])
    
    # Plot poissonian errorbars
    ax.errorbar(bin_centers, hist_entries, yerr=np.sqrt(hist_entries), color='black', fmt='none')
    
    # Plot mean, standard deviation and average uncertainty
    y_lims = ax.get_ylim() # Keep y-axis limits constant
    l_mean = ax.plot([p_avg,p_avg], [0,y_lims[1]],color='red', label="Mean of toys")
    l_std_min = ax.plot([p_avg-p_std,p_avg-p_std], [0,y_lims[1]],color='green', label="Std. of toys")
    l_std_max = ax.plot([p_avg+p_std,p_avg+p_std], [0,y_lims[1]],color='green')
    l_unc_min = ax.plot([p_avg-unc_avg[p],p_avg-unc_avg[p]], [0,y_lims[1]],color='magenta', label="Avg. fit unc.")
    l_unc_max = ax.plot([p_avg+unc_avg[p],p_avg+unc_avg[p]], [0,y_lims[1]],color='magenta')
    ax.set_ylim(y_lims) # Keep y-axis limits constant
    
    # Add legend
    ax.legend(title=result.par_names[p])
    
    fig.savefig("../output/hist_" + result.par_names[p] + ".pdf")
    print(result.par_names[p] + " avg: " + str(p_avg) + " std: " + str(p_std) + " unc: " + str(unc_avg[p]))

### Draw average correlation matrix

In [ ]:
# Create figure for plots and adjust height and width
fig_cor, ax_cor = plt.subplots()
fig_cor.set_figheight(8.5)
fig_cor.set_figwidth(10)

# Plot a 2D color plot (sharp, no interpolations, using PRGn color map)
im_cor = ax_cor.imshow(cor_matrix_avg,interpolation='none',cmap='PRGn')
cb_cor = fig_cor.colorbar(im_cor, ax=ax_cor) # Show what the colors mean
im_cor.set_clim(-1, 1); # Correlations are between -1 and +1

# We want to show all ticks...
ax_cor.set_xticks(np.arange(n_pars))
ax_cor.set_yticks(np.arange(n_pars))
# ... and label them with the respective list entries
ax_cor.set_xticklabels(result.par_names)
ax_cor.set_yticklabels(result.par_names)

# Rotate the tick labels and set their alignment.
plt.setp(ax_cor.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

ax_cor.set_title("Average correlation matrix")
fig_cor.tight_layout()
fig_cor.savefig("../output/hist_cor.pdf")

### Plot the chi-sq/ndof distribution

In [ ]:
# Degrees of freedom = #Bins - #FreeParameters (Here: Assume all fits have same ndof)
n_dof = result.fit_results[0].n_bins - result.fit_results[0].n_free_pars

norm_chi_sqs = chi_sqs/n_dof
norm_chi_sq_avg = np.average(norm_chi_sqs)

chi_sq_min = np.amin(norm_chi_sqs) # Smallest chi^2/ndof found
chi_sq_max = np.amax(norm_chi_sqs) # Largest chi^2/ndof found

# x-axis minimum and maximum
x_min_chi_sq = chi_sq_min - 0.1*(norm_chi_sq_avg-chi_sq_min) 
x_max_chi_sq = chi_sq_max + 0.1*(chi_sq_max-norm_chi_sq_avg)

# Plot histogram of all found chi^2 values
fig_chi_sq, ax_chi_sq = plt.subplots(tight_layout=True)
hist_chi_sq = ax_chi_sq.hist(norm_chi_sqs, range=(x_min_chi_sq,x_max_chi_sq), bins=15, histtype='step', color='black')
ax_chi_sq.set_xlabel(r"$\chi^2/ndof$")
ax_chi_sq.set_ylabel("#Toys")
ax_chi_sq.set_ylim([0,1.1*np.amax(hist_chi_sq[0])])

# Plot mean chi^2 of all toy fits
y_lims_chi_sq = ax_chi_sq.get_ylim()
l_mean_chi_sq = ax_chi_sq.plot([norm_chi_sq_avg,norm_chi_sq_avg], [0,y_lims_chi_sq[1]],color='red')
ax_chi_sq.set_ylim(y_lims_chi_sq)

# Box showing the number of degrees of freedom
ndof_str = r"$ndof$ = " + str(n_dof)
props = dict(boxstyle='round', facecolor='white')
ax_chi_sq.text(0.65, 0.85, ndof_str, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)

fig_chi_sq.savefig("../output/hist_chisq.pdf")
print(r"Average $\chi^2$: " + str(chi_sq_avg))
print(r"Average $\chi^2/ndof$: " + str(norm_chi_sq_avg))

### Plot cov. matrix calculation status statitics to detect if problems occured

In [ ]:
# Get covariance matrix calculation status for each fit
cov_status_arr = np.empty(n_fits)
for f in range(n_fits):
    cov_status_arr[f] = result.fit_results[f].cov_status
    
# Plot the status histogram
fig_cov_stat, ax_cov_stat = plt.subplots(tight_layout=True)
hist_cov_stat = ax_cov_stat.hist(cov_status_arr, range=(-1.5,3.5), bins=5, histtype='step', color='black')
ax_cov_stat.set_title("Cov. matr. calc. status")
ax_cov_stat.set_ylabel("#Toys")
ax_cov_stat.set_yscale('log') # Log scale to see when individuals go wrong

# Set appropriate y axis range
y_lims_cov_stat = ax_cov_stat.get_ylim()
ax_cov_stat.set_ylim(0.9, 1.2*np.amax(hist_cov_stat[0]))

# x axis labels with possible outcomes
cov_stat_ax_labels = ["PrEW output failure", "not calculated", "approximated", "made pos def", "accurate"]
ax_cov_stat.set_xticks(np.arange(-1, len(cov_stat_ax_labels)))
ax_cov_stat.set_xticklabels(cov_stat_ax_labels)

# Rotate the tick labels and set their alignment.
plt.setp(ax_cov_stat.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

fig_cov_stat.savefig("../output/hist_cov_status.pdf")

### Plot minimizer status statitics to detect if problems occured

In [10]:
min_status_arr = np.empty(n_fits)
for f in range(n_fits):
    min_status_arr[f] = result.fit_results[f].min_status
    
fig_min_stat, ax_min_stat = plt.subplots(tight_layout=True)
hist_min_stat = ax_min_stat.hist(min_status_arr, range=(-1.5,6.5), bins=8, histtype='step', color='black')
ax_min_stat.set_title("Minimizer status")
ax_min_stat.set_ylabel("#Toys")
ax_min_stat.set_yscale('log')

y_lims_min_stat = ax_min_stat.get_ylim()
ax_min_stat.set_ylim(0.9, 1.2*np.amax(hist_min_stat[0]))

# Write x-axis labels that tell problem exactly (corresponding to value from -1 to 6)
min_stat_ax_labels = ["PrEW output failure", "All good", "Cov-matr. made pos. def.", "Hesse not valid", "EDM above max", "Call limit reached", "Cov-matr. not pos. def.", "UNEXPECTED"]
ax_min_stat.set_xticks(np.arange(-1, len(min_stat_ax_labels)))
ax_min_stat.set_xticklabels(min_stat_ax_labels)

# Rotate the tick labels and set their alignment.
plt.setp(ax_min_stat.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

fig_min_stat.savefig("../output/hist_min_status.pdf")

### Plot number of fct calls and iterations for optimization

In [11]:
n_calls_arr = np.empty(n_fits)
n_iters_arr = np.empty(n_fits)
for f in range(n_fits):
    n_calls_arr[f] = result.fit_results[f].n_fct_calls
    n_iters_arr[f] = result.fit_results[f].n_iters

# Create the figure with the histograms
fig_n_stat, ax_n_stat = plt.subplots(tight_layout=True)
hist_n_calls_stat = ax_n_stat.hist(n_calls_arr, bins=15, histtype='step', color='blue', label="#FctCalls")
hist_n_iters_stat = ax_n_stat.hist(n_iters_arr, bins=15, histtype='step', color='red', label="#Iterations")
ax_n_stat.set_xlabel("N")
ax_n_stat.set_ylabel("#Toys")
ax_n_stat.set_ylim([0, 1.2*np.amax([np.amax(hist_n_calls_stat[0]),np.amax(hist_n_iters_stat[0])])])

# Add a legend 
ax_n_stat.legend()

fig_n_stat.savefig("../output/hist_n_stats.pdf")